<a href="https://colab.research.google.com/github/machine-perception-robotics-group/ImageProcessingGoogleColabNotebooks/blob/master/07_keypoint_detection_description.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07. キーポイントマッチング

講義で説明する画像処理の方法について，google colaboratoryを利用して演習する．
google colaboratoryは，クラウドで実行する Jupyter ノートブック環境である.
google coraboratoryについては，[ここ](https://www.tdi.co.jp/miso/google-colaboratory-gpu)や[ここ](https://www.codexa.net/how-to-use-google-colaboratory/)を参考にすること．

下記のプログラムを実行すると，SIFT特徴量のキーポイント検出と特徴量記述，キーポイントマッチングを実行する．

## 準備

SIFTを使用するために，下記のコマンドを実行してOpenCVのバージョンを指定して再インストールする．


In [ ]:
!pip install opencv-python==4.10.0.84
!pip install opencv-contrib-python==4.10.0.84

**注意:** 以下のプログラムでエラーが発生した場合は，上記のOpenCVの再インストールコマンドを実行した後，ページ上部のメニューバーから「ランタイム --> ランタイムを再起動」を選択し，実行し直してください．



プログラムの動作に必要なデータをダウンロードし，zipファイルを解凍する．
`!`で始まるコマンドはPythonではなく，Linux（Ubuntu）のコマンドを実行している．

In [ ]:
!wget -q https://raw.githubusercontent.com/machine-perception-robotics-group/ImageProcessingGoogleColabNotebooks/master/image1.zip
!unzip -q image1.zip
!ls
!ls ./image1/

## 画像の読み込みと表示
必要なパッケージをインポートし，画像を表示する．

`img2`はキーポイントマッチングのために，45度回転させる．

In [ ]:
import math
import cv2
import numpy as np
from matplotlib import pyplot as plt

img1 = cv2.imread('./image1/woman-g.jpg', 0)
img2 = cv2.imread('./image1/woman-t.jpg', 0)

# 画像を45度回転
h, w = img2.shape
trans = cv2.getRotationMatrix2D((int(h/2), int(w/2)), 45 , 1.0)
img2 = cv2.warpAffine(img2, trans, (h, w))

plt.imshow(img1, cmap="gray", clim=(0, 255))
plt.show()
plt.imshow(img2, cmap="gray", clim=(0, 255))
plt.show()

## SIFT

SIFTの処理は
1. キーポイント検出
2. 特徴記述

の2ステップから構成されている．

以下では，それぞれのステップの処理を実行し，その結果を確認する．
SIFTのキーポイント検出，特徴記述の処理は複雑なため，今回はOpenCVに含まれている関数を利用して行う．

### キーポイントの検出

まずはじめに，OpenCVの`SIFT_create`を呼び出す．
`SIFT_create`の呼び出しでは，まだキーポイント検出や特徴記述等は行なっておらず，SIFTの計算を行う際のパラメータ等を指定し，SIFTの計算を行うためのオブジェクト（クラスインスタンス）を作成している．
その後．`detect`関数にキーポイント検出をしたい画像を入力し，検出したキーポイント（`keypoints`）を返す．

`keypoints`を表示すると，

> [<KeyPoint 0x7f5157ce1630>, <KeyPoint 0x7f5157ce1540>, <KeyPoint 0x7f5157ce1690>, ... ]

のような文字が表示される．
これは，`detect`関数で検出されたキーポイントが，OpenCVの`KeyPoint`オブジェクトとしてリスト内に保存されいている．
一つ一つの`KeyPoint`オブジェクトには
* `pt`：キーポイントの座標
* `size`：キーポイントのスケール
* `angle`：キーポイントのオリエンテーション

などのような値が存在している．
（より詳細な情報を知りたい人は[OpenCVのドキュメント](https://docs.opencv.org/3.4/d2/d29/classcv_1_1KeyPoint.html)を参照すること）

ここで，`keypoints`のうち，0番目と1番目の`KeyPoint`の中身を表示すると，各キーポイントの座標やスケール等が数値として格納されていることがわかる．


In [ ]:
img = img1.copy()

# 画像を正規化
img = cv2.normalize(img, 0, 255, norm_type=cv2.NORM_MINMAX)

# SIFTを計算するための準備
sift = cv2.SIFT_create(nfeatures=0, nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6)
# SIFTの検出
keypoints = sift.detect(img)

print(keypoints[:10])
print(keypoints[0].pt, keypoints[0].size, keypoints[0].angle)
print(keypoints[1].pt, keypoints[1].size, keypoints[1].angle)

上で検出したキーポイントを画像へ描画する．

はじめに，描画用の画像を用意する．
この時，検出結果をカラーで表示するために，グレースケール画像からRGB画像へ変換しておく．


キーポイントの座標点の描画を行う．
for文でリスト内のキーポイントを一つづつ取り出し，座標点 (`pt`) の値に従い，点を描画する．
描画には画像中に円を描く`circle`関数を用いる．

次に，スケールを描画する．
スケールの描画には座標点と同様`circle`関数を使用する．
この時．円の半径を指定する第3引数をキーポイントのスケール (`size`) で指定することで，スケールを円の大きさで表示する．

最後に，オリエンテーションを描画する．
オリエンテーションの描画には，直線を描画する`line`関数を使用する．
この時，座標点からオリエンテーションの方向に従ってスケールの円に接するまでの直線を描画するが，`line`関数は直線の両端点を座標で指定する必要があるため．オリエンテーションから端点を算出する．



In [ ]:
# 描画用の画像を用意
img_sift1 = img1.copy()
# 検出結果をカラーで表示するため，グレースケールからRGB画像へ変換
img_sift1 = cv2.cvtColor(img_sift1, cv2.COLOR_GRAY2RGB)

# 座標点の描画
for k in keypoints:
  img_sift1 = cv2.circle(img_sift1, (int(k.pt[0]), int(k.pt[1])), 1, (0, 255, 0), -1)
plt.figure(figsize=(10, 8))
plt.imshow(img_sift1)
plt.show()

# スケールの描画
for k in keypoints:
  img_sift1 = cv2.circle(img_sift1, (int(k.pt[0]), int(k.pt[1])), int(k.size), (0, 255, 0), 1)
plt.figure(figsize=(10, 8))
plt.imshow(img_sift1)
plt.show()

# オリエンテーションの描画
for k in keypoints:
  ori_x = int(k.pt[0] + math.cos(math.radians(k.angle)) * k.size)
  ori_y = int(k.pt[1] + math.sin(math.radians(k.angle)) * k.size)
  img_sift1 = cv2.line(img_sift1, (int(k.pt[0]), int(k.pt[1])), (ori_x, ori_y), (0, 255, 0), 1)
plt.figure(figsize=(10, 8))
plt.imshow(img_sift1)
plt.show()

OpenCVの`drawKeypoints`関数を用いることでも，上と同様のキーポイントの描画を行うことができる．


In [ ]:
img_sift2 = cv2.drawKeypoints(img, keypoints, None, flags=4)
plt.figure(figsize=(10, 8))
plt.imshow(img_sift2)
plt.show()

### 特徴量の記述

次に，検出したキーポイントに対する，特徴量の記述を行う．
特徴量の記述にも，`SIFT_create`で作成したオブジェクトを使用する．
ここでは，上のプログラムで作成したオブジェクト`sift`を利用する．
`compute`関数に，画像と記述したい特徴量のキーポイントを入力することで，キーポイント`keypoints`と特徴量`desc`を返す．
ここで，`keypoints`は上で計算したものと同様である．

記述した特徴量を表示してみる．
`desc`は入れ子になったリストである．
1次元目が検出（記述）したキーポイントの数に対応しており．2次元目が各キーポイントの特徴量である．
SIFT特徴量は128次元であるため，2次元目の配列の長さも128となっている．






In [ ]:
# SIFT特徴量の記述
keypoints, desc = sift.compute(img, keypoints)

# 記述された特徴量（配列）のサイズの確認
print(desc)
print("the number of descriptors:", len(desc))

# 記述された特徴量の表示
print(desc[0])
print("length of each SIFT descriptor:", len(desc[0]))

## SIFTによるキーポイントマッチング

次に，SIFT特徴量を用いて，キーポイントのマッチングを行う．
ここでは，ページ上部で読み込んだ2枚の画像間でマッチングを行う．

まず，それぞれの画像からキーポイントを検出し，キーポイントに対応する特徴量を記述する．
ここでは，`detectAndCompute`という関数を使用しており，前述のキーポイント検出`detect`と特徴量記述`compute`を一度に行う関数となっている．



In [ ]:
img = img1.copy()
img_query = img2.copy()

# 画像を正規化
img = cv2.normalize(img, 0, 255, norm_type=cv2.NORM_MINMAX)
img_query = cv2.normalize(img_query, 0, 255, norm_type=cv2.NORM_MINMAX)

# SIFT特徴量の検出と記述
sift = cv2.SIFT_create(nfeatures=0, nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6)
kp, desc = sift.detectAndCompute(img, None)
kp_q, desc_q = sift.detectAndCompute(img_query, None)
print("the number of keypoints (original image):", len(kp))
print("the number of keypoints (query image):", len(kp_q))

次に，記述した特徴量間で距離を計算し，対応点を探索する．

距離の計算には，ユークリッド距離を使用するため，2つの特徴量間のユークリッド距離を計算するための関数`euclidean_dist`を定義する．
この関数では，2つの特徴量（`x1`, `x2`）を引数として入力し，そのユークリッド距離を返すものである．

In [ ]:
def euclidean_dist(x1, x2):
  if len(x1) != len(x2):
    print("ERROR: input data lengths are different.")
    exit(-1)
  x1 = np.array(x1, dtype=np.float32)
  x2 = np.array(x2, dtype=np.float32)
  dist = np.sqrt(np.sum(np.power(x1 - x2, 2)))
  return dist

上で定義した`euclidean_dist`関数を利用して，対応点の探索を行う．


まず，マッチングしたキーポイントの情報を格納するためのリスト`matched_keypoints`を用意する．

その後，一つ目の画像から記述された特徴量`desc`と二つ目の画像の特徴量`desc_q`をfor文で一つづつ取り出しながら，対応点探索を行う．

この時，取り出した特徴量に対応する番号（インデックス）を同時に扱うために，`enumerate`関数を利用する．
`enumerate`関数を用いてfor文を実行した場合，通常のfor文で取り出す変数とは別に`0, 1, 2, ...`というインデックスを同時に受け取ることが可能である．

まず，1枚目の画像のとある特徴量`d1`に対して，2枚目の画像の特徴量全てとのユークリッド距離を計算し，その値を`distances`に一時格納する．

その後，`distances`に格納されたユークリッド距離のうち，最も距離が小さい2つの特徴量の番号を`near_1st`, `near_2nd`に保存する．
この2つの特徴量に$d_{1}<d_{2} \times 0.6$の関係が成立する場合，1番目の特徴量は`d1`と対応しているとみなして．`matched_keypoints`にその特徴量のインデックスを保存する．

In [ ]:
matched_keypoints = []
for i, d1 in enumerate(desc):
  distances = []
  for j, d2 in enumerate(desc_q):
    distances.append(euclidean_dist(d1, d2))
  
  # 計算した距離を昇順に並べ替え
  near_order = np.argsort(distances)
  # 距離が1番目と2番目に小さいものを取得
  near_1st = distances[near_order[0]]
  near_2nd = distances[near_order[1]]

  if near_1st < (near_2nd * 0.6):
    matched_keypoints.append((i, near_order[0]))

print("the number of matched keypoints:", len(matched_keypoints))
print()

### 対応点の描画

対応点の探索が終了したら，対応点の描画を行う．

対応点を線で結んで表示するために，1枚の画像に2つの画像を表示する．

対応点の描画を行うために，`matched_keypoints`に格納されたインデックスのペアをfor文で一つづつ取り出して描画を行う．
各インデックスに対応するキーポイントの座標（`pt`）をリストから抽出し，`point1`, `point2`に格納する．
この時，`point2`は画像の右側に位置するように，x軸の値に1枚目の画像の横幅を加える．
その後．`point1`, `point2`それぞれの座標に点（円）を`circle`で描画し，`line`関数で直線を引くことで，対応点を描画する．
各対応点をわかりやすくするために，対応点ごとにランダムでRGB値を決定し，描画している．

In [ ]:
import random

h1, w1 = img.shape
h2, w2 = img_query.shape
# 2枚の画像を1つに表示するように配列を作成
img_match = np.zeros([h1, w1 + w2], dtype=np.uint8)
# 左側に1枚目をコピー
img_match[0:h1, 0:w1] = img.copy()
# 右側に2枚目をコピー
img_match[0:h2, w1:w1+w2] = img_query.copy()
# 検出結果をカラーで表示するため，グレースケールからRGB画像へ変換
img_match = cv2.cvtColor(img_match, cv2.COLOR_GRAY2RGB)

# 対応点の描画
for p1, p2 in matched_keypoints:
  point1 = (int(kp[p1].pt[0]), int(kp[p1].pt[1]))
  point2 = (int(w1 + kp_q[p2].pt[0]), int(kp_q[p2].pt[1]))
  color = (random.randint(30, 255), random.randint(30, 255), random.randint(30, 255))
  img_match = cv2.circle(img_match, point1, 3, color, 1)
  img_match = cv2.circle(img_match, point2, 3, color, 1)
  img_match = cv2.line(img_match, point1, point2, color, 1)

# 表示
plt.figure(figsize=(10, 8))
plt.imshow(img_match)
plt.show()

OpenCVの`knnMatch`関数と`drawMatchesKnn`関数を用いることでも同様の処理が可能である．



In [ ]:
bf = cv2.BFMatcher()
matches = bf.knnMatch(desc, desc_q, k=2)

# Apply ratio test
good = []
for m, n in matches:
    if m.distance < 0.6 * n.distance:
        good.append([m])

# cv2.drawMatchesKnn expects list of lists as matches.
draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   flags = 0)
img3 = cv2.drawMatchesKnn(img, kp, img_query ,kp_q, good, None, flags=2)

plt.figure(figsize=(10, 8))
plt.imshow(img3)
plt.show()

## 課題

* `SIFT_create`のパラメータを変更した場合に検出されるキーポイントがどのように変化するか確認すること
* どのような場所がキーポイントになっているか，考察すること
* どのような場所でキーポイントマッチングが成功または失敗しているか，考察すること
